# DATASCI S11 Final Project CBMS Dataset 
### By : Amiel James Bornales, Jonn Ralge Yuvallos

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import json
import folium
from folium.plugins import HeatMap

%matplotlib inline  

# The PNP Dataset
The PNP Dataset contains several categories of crimes, as well as the Degrees of Consummation of the said crimes. There are 3 Degrees of Consummation, 
namely Attempted, Frustrated. According the Article 6 of the Penal Code of the Philippines, the Degrees of Consumation can be defined as: 
* Attempted - When all elements of the felony is present but not the execution
* Frustrated - When all elements and execution is present but did not result in the desired result.
* Consumated - When all elements necessary for execution and accomplishment for a felony is present

In [2]:
df_pnp = pd.read_csv('data/PNP_data.csv')
df_pnp.head()

FileNotFoundError: File b'data/PNP_data.csv' does not exist

In [3]:
df_pnp_cols = df_pnp.iloc[0].tolist()
df_pnp_cols[0] = 'brgy'
df_pnp_cols

['brgy',
 'ATTEMPTED HOMICIDE(PERSONS)',
 'ATTEMPTED MURDER(PERSONS)',
 'ATTEMPTED PARRICIDE(PERSONS)',
 'ATTEMPTED RAPE (Art. 266-A RC & R.A.8353)(PERSONS)',
 'ATTEMPTED ROBBERY(PROPERTY)',
 'ATTEMPTED THEFT(PROPERTY)',
 'CONSUMATED ANTI-CARNAPPING ACT (R.A. 6539) MC(SPECIAL LAWS)',
 'CONSUMATED ANTI-CARNAPPING ACT (R.A. 6539) MV(SPECIAL LAWS)',
 'CONSUMATED HOMICIDE(PERSONS)',
 'CONSUMATED MURDER(PERSONS)',
 'CONSUMATED PHYSICAL INJURIES(PERSONS)',
 'CONSUMATED RAPE (Art. 266-A RC & R.A.8353)(PERSONS)',
 'CONSUMATED ROBBERY(PROPERTY)',
 'CONSUMATED THEFT(PROPERTY)',
 'FRUSTRATED HOMICIDE(PERSONS)',
 'FRUSTRATED MURDER(PERSONS)',
 'QUALIFIED THEFT(PROPERTY)',
 'SERIOUS PHYSICAL INJURIES(PERSONS)',
 'Grand Total']

In [4]:
# Adds column names instead of being the first row
df_pnp.columns = df_pnp_cols
df_pnp.drop(0,inplace = True)

#resets the index for the rows and drops the 'index' column that was generated
df_pnp = df_pnp.reset_index()
df_pnp.drop('index', axis = 1, inplace = True)

#drop the summary at the bottom
df_pnp.drop(df_pnp.index[188:], inplace= True)

#fills all the NaN with 0
df_pnp = df_pnp.fillna(0)

#convert data into int, since they're objects at the moment
for col in df_pnp_cols:
    if col != 'brgy':
        df_pnp[col] = pd.to_numeric(df_pnp[col], downcast = 'integer')
df_pnp.head()

,brgy,ATTEMPTED HOMICIDE(PERSONS),ATTEMPTED MURDER(PERSONS),ATTEMPTED PARRICIDE(PERSONS),ATTEMPTED RAPE (Art. 266-A RC & R.A.8353)(PERSONS),ATTEMPTED ROBBERY(PROPERTY),ATTEMPTED THEFT(PROPERTY),CONSUMATED ANTI-CARNAPPING ACT (R.A. 6539) MC(SPECIAL LAWS),CONSUMATED ANTI-CARNAPPING ACT (R.A. 6539) MV(SPECIAL LAWS),CONSUMATED HOMICIDE(PERSONS),CONSUMATED MURDER(PERSONS),CONSUMATED PHYSICAL INJURIES(PERSONS),CONSUMATED RAPE (Art. 266-A RC & R.A.8353)(PERSONS),CONSUMATED ROBBERY(PROPERTY),CONSUMATED THEFT(PROPERTY),FRUSTRATED HOMICIDE(PERSONS),FRUSTRATED MURDER(PERSONS),QUALIFIED THEFT(PROPERTY),SERIOUS PHYSICAL INJURIES(PERSONS),Grand Total
0,Barangay 1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1
1,Barangay 10,0,0,0,0,0,0,1,0,0,0,0,0,0,1,1,0,0,0,3
2,Barangay 100,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,2
3,Barangay 101,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,2
4,Barangay 103,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1


## Group the similar crime categories

In [5]:
df_pnp_cmp = pd.DataFrame(df_pnp['brgy'])
df_pnp_cmp['Attempted Murders'] = df_pnp['ATTEMPTED HOMICIDE(PERSONS)'] + df_pnp['ATTEMPTED MURDER(PERSONS)'] + df_pnp['ATTEMPTED PARRICIDE(PERSONS)']
df_pnp_cmp['Attempted Rape'] = df_pnp['ATTEMPTED RAPE (Art. 266-A RC & R.A.8353)(PERSONS)']
df_pnp_cmp['Attempted Robberies'] = df_pnp['ATTEMPTED ROBBERY(PROPERTY)'] + df_pnp['ATTEMPTED THEFT(PROPERTY)']
df_pnp_cmp['Consumated Carnapping'] = df_pnp['CONSUMATED ANTI-CARNAPPING ACT (R.A. 6539) MC(SPECIAL LAWS)'] + df_pnp['CONSUMATED ANTI-CARNAPPING ACT (R.A. 6539) MV(SPECIAL LAWS)']
df_pnp_cmp['Consumated Murder'] = df_pnp['CONSUMATED HOMICIDE(PERSONS)'] + df_pnp['CONSUMATED MURDER(PERSONS)']
df_pnp_cmp['Consumated Injuries'] = df_pnp['CONSUMATED PHYSICAL INJURIES(PERSONS)'] + df_pnp['SERIOUS PHYSICAL INJURIES(PERSONS)']
df_pnp_cmp['Consumated Rape'] = df_pnp[ 'CONSUMATED RAPE (Art. 266-A RC & R.A.8353)(PERSONS)']
df_pnp_cmp['Consumated Theft'] = df_pnp['CONSUMATED ROBBERY(PROPERTY)'] + df_pnp['CONSUMATED THEFT(PROPERTY)'] + df_pnp['QUALIFIED THEFT(PROPERTY)']
df_pnp_cmp['Frustrated Murder'] = df_pnp['FRUSTRATED HOMICIDE(PERSONS)'] + df_pnp['FRUSTRATED MURDER(PERSONS)']
df_pnp_cmp['Grand Total'] = df_pnp['Grand Total']
df_pnp_cmp.head()

,brgy,Attempted Murders,Attempted Rape,Attempted Robberies,Consumated Carnapping,Consumated Murder,Consumated Injuries,Consumated Rape,Consumated Theft,Frustrated Murder,Grand Total
0,Barangay 1,0,0,0,1,0,0,0,0,0,1
1,Barangay 10,0,0,0,1,0,0,0,1,1,3
2,Barangay 100,0,0,0,0,0,1,0,1,0,2
3,Barangay 101,1,0,0,0,0,0,0,1,0,2
4,Barangay 103,0,0,0,0,0,0,0,0,1,1


In [6]:
df_pnp_cmp.columns

Index(['brgy', 'Attempted Murders', 'Attempted Rape', 'Attempted Robberies',
       'Consumated Carnapping', 'Consumated Murder', 'Consumated Injuries',
       'Consumated Rape', 'Consumated Theft', 'Frustrated Murder',
       'Grand Total'],
      dtype='object')

### Function for generating the Maps

In [7]:
def create_map_pnp(dframe, col_n):
    geomap = folium.Map(location=[14.539070, 120.9938], width=600, height=600, zoom_start = 13)
    geomap.choropleth(geo_data = 'data/pasay_barangays.json', 
                 data = dframe,
                 columns = ['brgy', col_n],
                 key_on = 'feature.properties.NAME_3',
                 fill_color = 'YlOrRd',
                 highlight = True)
    return geomap

## HeatMap for the whole crime data
So we could have a general idea on which areas are prone to crime.

In [8]:
create_map_pnp(df_pnp_cmp, 'Grand Total')

D:\Program Files\Python\lib\site-packages\folium\folium.py:432: FutureWarning: The choropleth  method has been deprecated. Instead use the new Choropleth class, which has the same arguments. See the example notebook 'GeoJSON_and_choropleth' for how to do this.
  FutureWarning


Since Barangay 76 (Mall Of Asia area) contains over 30% of the crime data in the whole dataset (647 crimes out of 2006), the other
barangay's crime heatmap get affected, so we will be making a DataFrame that doesnt include Barangay 76 so as to make the heatmap more
detailed to the rest of the barangays.

In [9]:
df_pnp_no_moa = pd.DataFrame(df_pnp_cmp)
df_pnp_no_moa = df_pnp_no_moa[df_pnp_cmp['brgy']!='Barangay 76']

## Whole crime data without Barangay 76

In [10]:
create_map_pnp(df_pnp_no_moa, 'Grand Total')

# Generalized PNP Reports

### Attempted Murders

In [11]:
create_map_pnp(df_pnp_cmp,'Attempted Murders')

### Attempted Rape

In [12]:
create_map_pnp(df_pnp_cmp,'Attempted Rape')

### Attempted Robberies

In [13]:
create_map_pnp(df_pnp_cmp,'Attempted Robberies')

### Consumated Carnapping

In [14]:
create_map_pnp(df_pnp_cmp,'Consumated Carnapping')

### Consumated Murder

In [15]:
create_map_pnp(df_pnp_cmp,'Consumated Murder')

### Consumated Injuries

In [16]:
create_map_pnp(df_pnp_cmp,'Consumated Injuries')

### Consumated Rape

In [17]:
create_map_pnp(df_pnp_cmp,'Consumated Rape')

### Consumated Theft

In [18]:
create_map_pnp(df_pnp_cmp,'Consumated Theft')

### Consumated Theft (Without Brgy 76)
Since Barangay 76 has over 500 cases of theft, we decided to map another heatmap without it.

In [19]:
create_map_pnp(df_pnp_no_moa,'Consumated Theft')

### Frustrated Murder

In [20]:
create_map_pnp(df_pnp_cmp,'Frustrated Murder')

## Recommendations

* Increase the Police force in the areas where crime is more common

* Tell other criminals to not do crimes in Brgy 76 anymore, theres too much already lmao
